<a href="https://colab.research.google.com/github/DIPANJAN001/Dipanjanpatra/blob/master/StepFunc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [433]:
import numpy as np
import gym
from gym.utils import seeding
from gym.spaces import Space, Discrete, MultiDiscrete,  Box
from gym.spaces.space import Space
import numpy as np

In [434]:
class MyEnv(gym.Env):
    
      def __init__(self):
        
        self.action_space = Discrete(4)
        self.observation_space=Box(low=np.array([0]),high=np.array([100]),dtype=np.float32) 
        self.battery_efficiency=0.3
        self.battery_cap=30
        self.max_battery_capacity=50
        #self.reward, self.done, self.info= None, None, None
        #self.grid_import=0
        #self.grid_export=0
        self.info=None
      def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

      def _step(self,action):
        grid_import_=0
        grid_export_=0
        done=False
        load=np.random.randint(60,120)#df['demand']
        power=np.random.randint(60,120)#df['Solar_gen']
        net_load=max((load-power),0)
        net_rem=max(net_load-(self.battery_efficiency*self.battery_cap),0)
        info={}
        reward=0
        if action== 0:
          grid_export_=grid_export_+self.battery_cap+power-load-self.max_battery_capacity # grid export
          battery_chng=abs(self.battery_efficiency*self.max_battery_capacity)
          self.battery_cap=min(max(self.battery_efficiency*self.max_battery_capacity,0),50) # battery charge
          #reward=2
        if action== 1:
          battery_chng=abs(self.battery_efficiency*(power-load))
          self.battery_cap=min(max(self.battery_cap + self.battery_efficiency*(power-load),0),50)# only battery charge
          #reward=1
        if action== 2:
          self.grid_import_=net_rem #grid import
          battery_chng=abs(self.battery_efficiency*self.battery_cap)
          self.battery_cap=min(max(self.battery_cap-self.battery_efficiency*self.battery_cap,0),50)# battery discharge
          #reward=-2
        if action== 3:
          battery_chng=abs(self.battery_efficiency*(load-power))
          self.battery_cap=min(max(self.battery_cap-(load-power),0),50) #only battery discharge
          #reward=-1             
        #reward=(self.grid_export-self.grid_import)
        reward=grid_export_*8-grid_import_*10-(battery_chng*0.5)
        
        if(self.battery_cap<5):
              done=True
        
        return self.battery_cap,reward, done,info
        
      def _reset(self):
        self.battery_cap=30
        #self.done=False
        return self.battery_cap


In [435]:
env=MyEnv()

In [436]:
for e in range(50): #Episode
    #Play 20 episodes 
    score = 0
    observation = env._reset()
    #print(observation)
    done=False
    while not done:
        #env.render()
        action = env.action_space.sample()
        observation,reward,done,info = env._step(action)
        print(observation)
        score+=reward
        
        #print(info['load'],info['power'])
      
    print('episode:{} score:{}'.format(e,score))


21.0
15.0
10.5
15.0
26.7
15.0
10.5
7.35
15.0
15.0
12.6
18.6
11.400000000000002
15.0
10.5
0
episode:0 score:-1685.4250000000002
15.0
15.0
15.0
10.5
33.5
15.0
11.4
15.0
10.5
0
episode:1 score:-1604.55
0
episode:2 score:-7.05
41.4
27.6
27.0
18.9
41.9
15.0
10.5
16.2
0
episode:3 score:-277.99999999999994
15.0
15.0
15.0
24.6
17.220000000000002
12.054000000000002
15.0
0
episode:4 score:-1254.3410000000001
32.7
30.300000000000004
24.300000000000004
11.300000000000004
16.300000000000004
5.200000000000005
15.0
50
18
12.600000000000001
8.82
19.82
15.0
10.5
7.35
5.145
15.0
0
episode:5 score:-1382.0475
15.0
0
episode:6 score:13.5
12
8.4
5.880000000000001
4.1160000000000005
episode:7 score:-6.6419999999999995
32.1
22.47
50
49.1
34.370000000000005
7.3700000000000045
5.159000000000003
15.0
31.0
49.0
50
50
44.0
15.0
10.5
7.35
0
episode:8 score:-287.6885
21.0
14.7
13.5
0
episode:9 score:-15.75
15.0
15.0
18.6
15.0
20.0
30.0
15.0
4.0
episode:10 score:-542.9
21.0
15.0
0
episode:11 score:-115.2
15.0
15.0
10